In [2]:
import pandas as pd
import requests
from sqlalchemy import create_engine
import random
from datetime import datetime
import json
from postgre_cred import database_url

In [3]:
tech_cat_data = requests.get('https://veli.store/_next/data/O6jvrOse3zlGpiC7AWBmz/ka/category/teqnika/774.json?type=teqnika&type=774')

In [5]:
tech_cat_data = tech_cat_data.json()
tech_cat_data = tech_cat_data['pageProps']['data']['categories']
tech_cat_data

[{'id': 59,
  'headline': 'მობილურები & აქსესუარები',
  'image': None,
  'icon': 'https://media.veli.store/media/product_category/tech_light_blue.svg',
  'meta_description': 'აღმოაჩინე კატეგორია - მობილურები & აქსესუარები, მიიღე შეკვეთა 3 საათში',
  'meta_title': None,
  'full_slug': 'teqnika/mobilurebi-aqsesuarebi',
  'canonical_url': None},
 {'id': 111,
  'headline': 'კომპიუტერული ტექნიკა',
  'image': None,
  'icon': 'https://media.veli.store/media/product_category/tc.svg',
  'meta_description': 'შეიძინე კომპიუტერული ტექნიკა ველიზე შენთვის ან შენი ბიზნესისთვის, მიიღე შენაძენი 3 საათში უფასოდ, ნებისმიერ მისამართზე.',
  'meta_title': None,
  'full_slug': 'teqnika/kompiuteruli-teqnika',
  'canonical_url': None},
 {'id': 775,
  'headline': 'პლანშეტები & აქსესუარები',
  'image': None,
  'icon': 'https://media.veli.store/media/product_category/tech_light_green.svg',
  'meta_description': 'აღმოაჩინე პლანშეტები და აქსესუარები VELI.store-ზე. შეიძინე სასურველი ნივთები და მიიღე შენაძენი 3 საათშ

In [6]:
tech_cat_df = pd.DataFrame(tech_cat_data)
tech_cat_df = tech_cat_df[['id', 'full_slug']]
tech_cat_df

,id,full_slug
0,59,teqnika/mobilurebi-aqsesuarebi
1,111,teqnika/kompiuteruli-teqnika
2,775,teqnika/planshetebi-aqsesuarebi
3,2667,teqnika/televizorebi-aqsesuarebi
4,7018,teqnika/geimingi
5,5281,teqnika/mskhvili-teqnika
6,829,teqnika/smart-gajetebi
7,8436,teqnika/foto-video
8,776,teqnika/audioteqnika
9,1233,teqnika/tavis-movla


In [22]:
full_slug = 'teqnika/mobilurebi-aqsesuarebi/qeisebi-ekranis-damcavebi/ekranis-damcavebi'
slug_parts = full_slug.split('/')
slug_parts = list(map(lambda x: f'type={x}&', slug_parts))
''.join(slug_parts)

'type=teqnika&type=mobilurebi-aqsesuarebi&type=qeisebi-ekranis-damcavebi&type=ekranis-damcavebi&'

In [25]:
def generate_url(full_slug: str, cat_id: int):
    url = f'https://veli.store/_next/data/O6jvrOse3zlGpiC7AWBmz/ka/category/{full_slug}/{cat_id}.json?'
    slug_parts = full_slug.split('/')
    slug_parts = list(map(lambda x: f'type={x}&', slug_parts))
    slug_parts = ''.join(slug_parts)
    url = f'{url}{slug_parts}type={cat_id}'
    return url

In [27]:
tech_cat_df['url'] = tech_cat_df.apply(lambda row: generate_url(row['full_slug'], row['id']), axis=1)
tech_cat_df['url'][3]

'https://veli.store/_next/data/O6jvrOse3zlGpiC7AWBmz/ka/category/teqnika/televizorebi-aqsesuarebi/2667.json?type=teqnika&type=televizorebi-aqsesuarebi&type=2667'

In [28]:
tech_sub_cat_data = requests.get(tech_cat_df['url'][0])    
tech_sub_cat_data = tech_sub_cat_data.json()
tech_sub_cat_data = tech_sub_cat_data['pageProps']['data']['categories']
tech_sub_cat_df = pd.DataFrame(tech_sub_cat_data)
tech_sub_cat_df = tech_sub_cat_df[['id', 'full_slug']]
tech_sub_cat_df['url'] = tech_sub_cat_df.apply(lambda row: generate_url(row['full_slug'], row['id']), axis=1)
tech_sub_cat_df['url'][0]

'https://veli.store/_next/data/O6jvrOse3zlGpiC7AWBmz/ka/category/teqnika/mobilurebi-aqsesuarebi/mobiluri-telefonebi/60.json?type=teqnika&type=mobilurebi-aqsesuarebi&type=mobiluri-telefonebi&type=60'

In [17]:
page = 1
products = []
while True:
    resp = requests.get(f'https://veli.store/_next/data/O6jvrOse3zlGpiC7AWBmz/ka/category/teqnika/mobilurebi-aqsesuarebi/mobiluri-telefonebi/60.json?type=teqnika&type=mobilurebi-aqsesuarebi&type=mobiluri-telefonebi&type=60&page_size=1000&page={page}')
    print(f'on page {page} status code is {resp.status_code}')
    resp = resp.json()
    current_products = resp['pageProps']['data']['products']
    if not len(current_products):
        break
    products.extend(current_products)
    page += 1


df = pd.DataFrame(products)

df

on page 1 status code is 200
on page 2 status code is 200
on page 3 status code is 200
on page 4 status code is 200


,id,headline,rating,image,stock,sku,slug,badges,brand,visible
0,141139,Oppo A78 CPH2565 8 GB 256 GB Black,0,https://media.veli.store/media/__sized__/produ...,"{'start_price': 599, 'price': 489}",6932169332491,oppo-a78-cph2565-8-gb-256-gb-black,[],ოპპო,True
1,166370,Apple iPhone 16 Pro Max 8 GB 512 GB Natural Ti...,0,https://media.veli.store/media/__sized__/produ...,"{'start_price': 5299, 'price': 4899}",195949807022,apple-iphone-16-pro-max-8-gb-512-gb-natural-ti...,[],ეფლი,True
2,43259,Apple iPhone 14 6 GB 128 GB Purple,0,https://media.veli.store/media/__sized__/produ...,"{'start_price': 2099, 'price': 1999}",194253408741,apple-iphone-14-6-gb-128-gb-purple,"[{'headline': 'ველი გირჩევს', 'icon': 'https:/...",ეფლი,True
3,112758,Samsung Galaxy S24 8 GB 128 GB 5G Dual Sim Black,0,https://media.veli.store/media/__sized__/produ...,"{'start_price': 2139, 'price': 1899}",8806095296951,samsung-galaxy-s24-8-gb-128-gb-5g-dual-sim-black,"[{'headline': 'ველი გირჩევს', 'icon': 'https:/...",სამსუნგი,True
4,189219,Samsung Galaxy S24 FE 5G S721B 8 GB 256 GB Duo...,0,https://media.veli.store/media/__sized__/produ...,"{'start_price': 1899, 'price': 1799}",8806095749471,samsung-galaxy-s24-fe-5g-s721b-8-gb-256-gb-duo...,[],სამსუნგი,True
...,...,...,...,...,...,...,...,...,...,...
83,116030,Samsung Galaxy A05s 4 GB 128 GB Black,0,https://media.veli.store/media/__sized__/produ...,"{'start_price': 359, 'price': 359}",8806095273518,samsung-galaxy-a05s-4-gb-128-gb-black,[],სამსუნგი,True
84,116028,Samsung Galaxy A05s 4 GB 64 GB Silver,0,https://media.veli.store/media/__sized__/produ...,"{'start_price': 299, 'price': 299}",8806095273464,samsung-galaxy-a05s-4-gb-64-gb-silver,[],სამსუნგი,True
85,116026,Samsung Galaxy A256E A25 6 GB 128 GB 5G Duos D...,0,https://media.veli.store/media/__sized__/produ...,"{'start_price': 819, 'price': 819}",8806095368191,samsung-galaxy-a256e-a25-6-gb-128-gb-5g-duos-d...,"[{'headline': 'ველი გირჩევს', 'icon': 'https:/...",სამსუნგი,True
86,116015,Samsung Galaxy A05s 4 GB 64 GB Black,0,https://media.veli.store/media/__sized__/produ...,"{'start_price': 299, 'price': 299}",8806095273556,samsung-galaxy-a05s-4-gb-64-gb-black,[],სამსუნგი,True


In [18]:
products

[{'id': 141139,
  'headline': 'Oppo A78 CPH2565 8 GB 256 GB Black',
  'rating': 0,
  'image': 'https://media.veli.store/media/__sized__/product/CPH2565-thumbnail-200x200-95.jpg',
  'stock': {'start_price': 599, 'price': 489},
  'sku': '6932169332491',
  'slug': 'oppo-a78-cph2565-8-gb-256-gb-black',
  'badges': [],
  'brand': 'ოპპო',
  'visible': True},
 {'id': 166370,
  'headline': 'Apple iPhone 16 Pro Max 8 GB 512 GB Natural Titanium',
  'rating': 0,
  'image': 'https://media.veli.store/media/__sized__/product/16promaxNatural_Titanium.jpg-0-thumbnail-200x200-95.jpg',
  'stock': {'start_price': 5299, 'price': 4899},
  'sku': '195949807022',
  'slug': 'apple-iphone-16-pro-max-8-gb-512-gb-natural-titanium',
  'badges': [],
  'brand': 'ეფლი',
  'visible': True},
 {'id': 43259,
  'headline': 'Apple iPhone 14 6 GB 128 GB Purple',
  'rating': 0,
  'image': 'https://media.veli.store/media/__sized__/product/Apple_iPhone_14_Plus_6_GB_128_GB_purple_ashasdhaf-thumbnail-200x200.png',
  'stock': {'